In [1]:
import gc
import numpy as np
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import utils, segmentation, AL, presentation
from tqdm.auto import tqdm

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:1


In [3]:
ds_full = segmentation.SegmentationDataset('./cut_stenoses_data/train/', 2)
ds_test = segmentation.SegmentationDataset('./cut_stenoses_data/test/', 2)

batch_size = 8
dl_full = torch.utils.data.DataLoader(ds_full, batch_size=batch_size, shuffle=False, drop_last=False)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [4]:
# w = 0.
# for imgs, masks in dl_full:
#     w += masks.sum() / (masks.numel() * len(dl_full))
# loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1 / (1-w), 1/ w])).to(device)
loss_fn = nn.CrossEntropyLoss()

In [5]:
model = presentation.create_model().to(device)
optimizer = presentation.create_optimizer(model)

In [6]:
N = len(ds_full)
N_step = int(N*0.1)
N, N_step

(598, 59)

In [7]:
idxs = list(range(N))
np.random.seed(0)
np.random.shuffle(idxs)

In [8]:
history = []
for n_iter in range(10):
    print(f'>>>>>>>>>>>> Iter {n_iter}')
    del model
    del optimizer
    gc.collect()
    labeled_idxs = idxs[:N_step*(n_iter+1)]
    history.append(labeled_idxs)
    ds_train = AL.PartialDs(ds_full, labeled_idxs)
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, drop_last=True)

    model = presentation.create_model().to(device)
    optimizer = presentation.create_optimizer(model)

    num_epochs = 50
    for epoch in tqdm(range(num_epochs), desc='Epoch'):
        trn_loss = utils.train(dl_train, loss_fn, model, optimizer)
        val_loss, val_IoUs = segmentation.evaluate(dl_test, loss_fn, model)
        print(epoch, f'{trn_loss:.4f}', f'{val_loss:.4f}', val_IoUs)
    print()

>>>>>>>>>>>> Iter 0


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

0 0.2553 0.3361 tensor([0.])
1 0.0224 0.0283 tensor([0.])
2 0.0156 0.0301 tensor([0.])
3 0.0158 0.0235 tensor([0.])
4 0.0101 0.0232 tensor([0.])
5 0.0087 0.0230 tensor([0.])
6 0.0093 0.0241 tensor([0.])
7 0.0082 0.0220 tensor([0.])
8 0.0076 0.0239 tensor([0.])
9 0.0075 0.0260 tensor([0.])
10 0.0075 0.0244 tensor([0.])
11 0.0071 0.0231 tensor([0.])
12 0.0070 0.0255 tensor([0.])
13 0.0070 0.0248 tensor([0.])
14 0.0068 0.0230 tensor([0.])
15 0.0067 0.0263 tensor([0.])
16 0.0065 0.0264 tensor([0.])
17 0.0064 0.0258 tensor([0.])
18 0.0064 0.0269 tensor([0.])
19 0.0062 0.0277 tensor([0.])
20 0.0061 0.0269 tensor([0.])
21 0.0061 0.0277 tensor([0.])
22 0.0060 0.0250 tensor([0.])
23 0.0059 0.0286 tensor([0.])
24 0.0057 0.0277 tensor([0.])
25 0.0057 0.0291 tensor([0.])
26 0.0057 0.0279 tensor([0.])
27 0.0057 0.0274 tensor([0.])
28 0.0055 0.0273 tensor([0.])
29 0.0055 0.0262 tensor([0.])
30 0.0054 0.0281 tensor([0.])
31 0.0053 0.0284 tensor([0.])
32 0.0054 0.0278 tensor([0.])
33 0.0053 0.0278 ten

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

0 0.0660 0.0317 tensor([0.])
1 0.0169 0.0244 tensor([0.])
2 0.0114 0.0197 tensor([0.])
3 0.0092 0.0160 tensor([0.])
4 0.0077 0.0175 tensor([0.])
5 0.0071 0.0179 tensor([0.])
6 0.0065 0.0224 tensor([0.])
7 0.0065 0.0184 tensor([0.])
8 0.0061 0.0157 tensor([0.])
9 0.0059 0.0170 tensor([0.])
10 0.0057 0.0185 tensor([0.])
11 0.0054 0.0200 tensor([0.])
12 0.0054 0.0186 tensor([0.])
13 0.0052 0.0197 tensor([0.])
14 0.0050 0.0193 tensor([5.9396e-05])
15 0.0049 0.0199 tensor([0.0009])
16 0.0048 0.0166 tensor([0.2807])
17 0.0047 0.0167 tensor([0.3189])
18 0.0047 0.0159 tensor([0.3657])
19 0.0046 0.0195 tensor([0.2916])
20 0.0045 0.0186 tensor([0.3121])
21 0.0044 0.0189 tensor([0.3442])
22 0.0043 0.0148 tensor([0.4433])
23 0.0042 0.0195 tensor([0.3426])
24 0.0042 0.0190 tensor([0.3631])
25 0.0042 0.0162 tensor([0.4255])
26 0.0040 0.0202 tensor([0.3488])
27 0.0040 0.0181 tensor([0.3952])
28 0.0039 0.0168 tensor([0.4266])
29 0.0038 0.0197 tensor([0.3868])
30 0.0038 0.0181 tensor([0.4118])
31 0.003

...

0 0.0329 0.0190 tensor([0.])
1 0.0070 0.0198 tensor([0.])
2 0.0061 0.0192 tensor([0.])
3 0.0055 0.0174 tensor([0.])
4 0.0050 0.0184 tensor([6.2154e-05])
5 0.0046 0.0182 tensor([0.3817])
6 0.0043 0.0188 tensor([0.4158])
7 0.0040 0.0145 tensor([0.4930])
8 0.0038 0.0169 tensor([0.4778])
9 0.0035 0.0163 tensor([0.4904])
10 0.0034 0.0183 tensor([0.4829])
11 0.0032 0.0158 tensor([0.5189])
12 0.0030 0.0177 tensor([0.5020])
13 0.0029 0.0182 tensor([0.5011])
14 0.0028 0.0191 tensor([0.5006])
15 0.0026 0.0183 tensor([0.5103])
16 0.0025 0.0176 tensor([0.5252])
17 0.0024 0.0200 tensor([0.4972])
18 0.0023 0.0159 tensor([0.5405])
19 0.0022 0.0181 tensor([0.5328])
20 0.0022 0.0178 tensor([0.5363])
21 0.0021 0.0160 tensor([0.5557])
22 0.0020 0.0178 tensor([0.5423])
23 0.0019 0.0181 tensor([0.5428])
24 0.0019 0.0187 tensor([0.5408])
25 0.0018 0.0179 tensor([0.5450])
26 0.0018 0.0184 tensor([0.5453])
27 0.0017 0.0171 tensor([0.5529])
28 0.0017 0.0186 tensor([0.5477])
29 0.0016 0.0186 tensor([0.5557])
30

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

0 0.0219 0.0238 tensor([0.])
1 0.0068 0.0187 tensor([0.])
2 0.0057 0.0168 tensor([5.9431e-05])
3 0.0051 0.0162 tensor([0.3158])
4 0.0046 0.0149 tensor([0.3878])
5 0.0042 0.0167 tensor([0.3863])
6 0.0039 0.0170 tensor([0.4178])
7 0.0037 0.0181 tensor([0.4177])
8 0.0034 0.0160 tensor([0.4506])
9 0.0033 0.0155 tensor([0.4656])
10 0.0030 0.0140 tensor([0.4974])
11 0.0029 0.0145 tensor([0.4934])
12 0.0028 0.0164 tensor([0.4747])
13 0.0026 0.0147 tensor([0.5012])
14 0.0025 0.0160 tensor([0.4873])
15 0.0024 0.0151 tensor([0.5067])
16 0.0023 0.0153 tensor([0.5181])
17 0.0022 0.0142 tensor([0.5358])
18 0.0022 0.0142 tensor([0.5381])
19 0.0021 0.0140 tensor([0.5402])
20 0.0020 0.0148 tensor([0.5401])
21 0.0019 0.0138 tensor([0.5487])
22 0.0018 0.0153 tensor([0.5392])
23 0.0018 0.0122 tensor([0.5831])
24 0.0017 0.0151 tensor([0.5430])
25 0.0017 0.0141 tensor([0.5607])
26 0.0016 0.0149 tensor([0.5536])
27 0.0015 0.0138 tensor([0.5685])
28 0.0015 0.0099 tensor([0.6192])
29 0.0015 0.0136 tensor([0.5

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

0 0.0263 0.0174 tensor([0.])
1 0.0067 0.0178 tensor([0.])
2 0.0057 0.0191 tensor([0.])
3 0.0051 0.0161 tensor([0.0122])
4 0.0046 0.0158 tensor([0.4458])
5 0.0042 0.0184 tensor([0.4480])
6 0.0039 0.0162 tensor([0.4963])
7 0.0036 0.0159 tensor([0.5122])
8 0.0033 0.0192 tensor([0.4781])
9 0.0032 0.0154 tensor([0.5321])
10 0.0029 0.0178 tensor([0.5153])
11 0.0028 0.0189 tensor([0.5041])
12 0.0027 0.0187 tensor([0.5153])
13 0.0025 0.0185 tensor([0.5223])
14 0.0024 0.0187 tensor([0.5249])
15 0.0023 0.0171 tensor([0.5474])
16 0.0022 0.0141 tensor([0.5752])
17 0.0021 0.0165 tensor([0.5549])
18 0.0020 0.0183 tensor([0.5428])
19 0.0020 0.0168 tensor([0.5523])
20 0.0019 0.0164 tensor([0.5645])
21 0.0018 0.0183 tensor([0.5488])
22 0.0018 0.0172 tensor([0.5659])
23 0.0017 0.0166 tensor([0.5667])
24 0.0016 0.0174 tensor([0.5693])
25 0.0016 0.0178 tensor([0.5667])
26 0.0016 0.0176 tensor([0.5716])
27 0.0015 0.0178 tensor([0.5804])
28 0.0015 0.0141 tensor([0.5984])
29 0.0014 0.0183 tensor([0.5688])
30